# Setup

In [1]:
# -*- coding: utf-8 -*-

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import sklearn as sk
import pickle as pkl

---
# Data

### About the dataset

Abalone
+ Sex / nominal / -- / M, F, and I (infant) 
+ Length / continuous / mm / Longest shell measurement 
+ Diameter	/ continuous / mm / perpendicular to length 
+ Height / continuous / mm / with meat in shell 
+ Whole weight / continuous / grams / whole abalone 
+ Shucked weight / continuous	/ grams / weight of meat 
+ Viscera weight / continuous / grams / gut weight (after bleeding) 
+ Shell weight / continuous / grams / after being dried 
+ Rings / integer / -- / +1.5 gives the age in years 

In [3]:
# variable names
names = [
    'sex',
    'length',
    'diameter',
    'height',
    'whole_weight',
    'shucked_weight',
    'viscera_weight',
    'shell_weight',
    'rings']

# reading dataset
df = pd.read_csv('data/abalone.data', header=None, names=names)

# building prediction target
df['target'] = (df['rings'] >= 10).astype(int)
df = df.drop('rings', axis=1)

In [4]:
df.head()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,target
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,1
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,0
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,0
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,1
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,0


---

# Feature Preparation

In [5]:
# seperating target from features
y = np.array(df['target'])
X = df.drop('target', axis=1)

In [6]:
# shuffling and splitting data into training and test sets
from sklearn.model_selection import train_test_split

SPLIT = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=SPLIT, random_state=42)

comment

In [7]:
# one-hot encoding categorical features
CATEGORICALS = ['sex']
DUMMIES = {
    'sex':['M','F','I']
}

def dummy_encode(in_df, dummies):
    out_df = in_df.copy()
    
    for feature, values in dummies.items():
        for value in values:
            dummy_name = '{}__{}'.format(feature, value)
            out_df[dummy_name] = (out_df[feature] == value).astype(int)
            
        del out_df[feature]
        # print('Dummy-encoded feature\t\t{}'.format(feature))
    return out_df
        
X_train = dummy_encode(in_df=X_train, dummies=DUMMIES)

X_test = dummy_encode(in_df=X_test, dummies=DUMMIES)

Dummy-encoded feature		sex
Dummy-encoded feature		sex


comment

In [8]:
# rescaling numerical features
NUMERICS = ['length','diameter','height','whole_weight','shucked_weight','viscera_weight','shell_weight']
BOUNDARIES = {
    'length': (0.075000, 0.815000),
    'diameter': (0.055000, 0.650000),
    'height': (0.000000, 1.130000),
    'whole_weight': (0.002000, 2.825500),
    'shucked_weight': (0.001000, 1.488000),
    'viscera_weight': (0.000500, 0.760000),
    'shell_weight': (0.001500, 1.005000)
}

def minmax_scale(in_df, boundaries):
    out_df = in_df.copy()
    
    for feature, (min_val, max_val) in boundaries.items():      
        col_name = '{}__norm'.format(feature)
        
        out_df[col_name] = round((out_df[feature] - min_val)/(max_val - min_val),3)
        out_df.loc[out_df[col_name] < 0, col_name] = 0
        out_df.loc[out_df[col_name] > 1, col_name] = 1
            
        del out_df[feature]
        # print('MinMax Scaled feature\t\t{}'.format(feature))
    return out_df
        
X_train = minmax_scale(in_df=X_train, boundaries=BOUNDARIES)

X_test = minmax_scale(in_df=X_test, boundaries=BOUNDARIES)

MinMax Scaled feature		diameter
MinMax Scaled feature		viscera_weight
MinMax Scaled feature		shucked_weight
MinMax Scaled feature		height
MinMax Scaled feature		shell_weight
MinMax Scaled feature		length
MinMax Scaled feature		whole_weight
MinMax Scaled feature		diameter
MinMax Scaled feature		viscera_weight
MinMax Scaled feature		shucked_weight
MinMax Scaled feature		height
MinMax Scaled feature		shell_weight
MinMax Scaled feature		length
MinMax Scaled feature		whole_weight


In [9]:
X_train.head()

,sex__M,sex__F,sex__I,diameter__norm,viscera_weight__norm,shucked_weight__norm,height__norm,shell_weight__norm,length__norm,whole_weight__norm
1593,0,0,1,0.546,0.209,0.175,0.119,0.173,0.608,0.217
111,1,0,0,0.513,0.140,0.115,0.093,0.173,0.527,0.152
3271,1,0,0,0.622,0.161,0.199,0.137,0.253,0.601,0.273
1089,0,0,1,0.462,0.103,0.125,0.093,0.096,0.507,0.131
2918,0,0,1,0.655,0.267,0.299,0.119,0.251,0.709,0.325


---
# Training

In [10]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(
    n_estimators=100, # number of trees
    n_jobs=-1, # parallelization
    random_state=1337, # random seed
    max_depth=10, # maximum tree depth
    min_samples_leaf=10
)

In [11]:
%time model = clf.fit(X_train, y_train)

CPU times: user 388 ms, sys: 20 ms, total: 408 ms
Wall time: 296 ms


---
# Evaluation

In [12]:
# computing ROC AUC over training set
train_auc = sk.metrics.roc_auc_score(y_train, model.predict(X_train))
print('Training ROC AUC:\t', round(train_auc, 3))

Training ROC AUC:	 0.85


In [13]:
# computing ROC AUC over test set
test_auc = sk.metrics.roc_auc_score(y_test, model.predict(X_test))
print('Test ROC AUC:\t\t', round(test_auc, 3))

Test ROC AUC:		 0.788


---
# Storing Model

In [14]:
pkl.dump(model, open('pickles/model_v1.pkl','wb'))

---
# Loading Model

In [15]:
m = pkl.load(open('pickles/model.pkl','rb'))
m

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=10,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False,
            random_state=1337, verbose=0, warm_start=False)

### All good. Let's build this bad boy into an API!